In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df_source = pd.read_csv('ReccomendationEngineSource.csv')
df_source.head()

,Title,Author,Narrator,Series,Genres
0,HereticalFishingBook1,HaylockJobson,HeathMiller,HereticalFishing1,SliceofLifeFishingFantasyHumerousLitRPG
1,TheDungeonAnarchistsCookbook,MattDinniman,JeffHays,DungeonCrawlerCarl3,LitRPGScienceFictionFantasyActionAdventureHume...
2,DungeonCrawlerCarl,MattDinniman,JeffHays,DungeonCrawlerCarl1,LitRPGScienceFictionFantasyHumerousActionAdven...
3,CarlsDoomsdayScenario,MattDinniman,JeffHays,DungeonCrawlerCarl2,LitRPGScienceFictionFantasyHumerousActionAdven...
4,HereticalFishing2ACozyGuidetoAnnoyingtheCultsO...,HaylockJobson,HeathMiller,HereticalFishing2,FantasyFishingHumerousLitRPGSliceofLife


In [3]:
df_source.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223 entries, 0 to 222
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Title     223 non-null    object
 1   Author    223 non-null    object
 2   Narrator  223 non-null    object
 3   Series    218 non-null    object
 4   Genres    223 non-null    object
dtypes: object(5)
memory usage: 8.8+ KB


In [4]:
df_source[df_source.duplicated(['Title'], keep=False)]

,Title,Author,Narrator,Series,Genres
50,Oathbringer,BrandonSanderson,GraphicAudio,TheStormlightArchivesGraphicAudio3,ActionAdventureEpicFantasy
51,RhythmofWar,BrandonSanderson,GraphicAudio,TheStormlightArchivesGraphicAudio4,ActionAdventureEpicFantasy
74,Oathbringer,BrandonSanderson,MichaelKramerKateReading,TheStormlightArchiveStandard3,ActionAdventureEpicFantasy
75,RhythmofWar,BrandonSanderson,KateReadingMichaelKramer,TheStormlightArchiveStandard4,FantasyEpicActionAdventure
82,TheAlloyofLaw,BrandonSanderson,MichaelKramer,Mistborn4,FantasyDramaWestern
85,TheAlloyofLaw,BrandonSanderson,MichaelKramer,Mistborn7,WesternFantasyEpic


In [5]:
df_cleaned = df_source.drop_duplicates(subset='Title')

In [6]:
df_cleaned['Title'] = df_cleaned['Title'].str.lower()
df_cleaned['Author'] = df_cleaned['Author'].str.lower()
df_cleaned['Narrator'] = df_cleaned['Narrator'].str.lower()
df_cleaned['Series'] = df_cleaned['Series'].str.lower()
df_cleaned['Genres'] = df_cleaned['Genres'].str.lower()
df_cleaned

C:\Users\chloe\AppData\Local\Temp\ipykernel_11720\2183843487.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Title'] = df_cleaned['Title'].str.lower()
C:\Users\chloe\AppData\Local\Temp\ipykernel_11720\2183843487.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Author'] = df_cleaned['Author'].str.lower()
C:\Users\chloe\AppData\Local\Temp\ipykernel_11720\2183843487.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

,Title,Author,Narrator,Series,Genres
0,hereticalfishingbook1,haylockjobson,heathmiller,hereticalfishing1,sliceoflifefishingfantasyhumerouslitrpg
1,thedungeonanarchistscookbook,mattdinniman,jeffhays,dungeoncrawlercarl3,litrpgsciencefictionfantasyactionadventurehume...
2,dungeoncrawlercarl,mattdinniman,jeffhays,dungeoncrawlercarl1,litrpgsciencefictionfantasyhumerousactionadven...
3,carlsdoomsdayscenario,mattdinniman,jeffhays,dungeoncrawlercarl2,litrpgsciencefictionfantasyhumerousactionadven...
4,hereticalfishing2acozyguidetoannoyingthecultso...,haylockjobson,heathmiller,hereticalfishing2,fantasyfishinghumerouslitrpgsliceoflife
...,...,...,...,...,...
218,somewherebeyondthesea,tjklune,danielhenning,ceruleanchronicles2,lgbtqyoungadultromancefantasy
219,thehouseintheceruleansea,tjklune,danielhenning,ceruleanchronicles1,lgbtqafantasyromanceyoungadult
220,starwarsheirtotheempire,timothyzahn,marcthompson,starwarsthethrawntrilogy1,actionadventureepicalienssciencefiction
221,starwarsthelastcommand,timothyzahn,marcthompson,starwarsthethrawntrilogy3,sciencefictionepicactionadventurealiensspaceop...


In [8]:
df_combined = df_cleaned
df_combined['data'] = df_combined[df_combined.columns[1:]].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)

print(df_combined['data'].head())

0    haylockjobson heathmiller hereticalfishing1 sl...
1    mattdinniman jeffhays dungeoncrawlercarl3 litr...
2    mattdinniman jeffhays dungeoncrawlercarl1 litr...
3    mattdinniman jeffhays dungeoncrawlercarl2 litr...
4    haylockjobson heathmiller hereticalfishing2 fa...
Name: data, dtype: object


C:\Users\chloe\AppData\Local\Temp\ipykernel_11720\3806835771.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined['data'] = df_combined[df_combined.columns[1:]].apply(


In [ ]:
vectorizer = CountVectorizer()
vectorized = vectorizer.fit_transform(df_combined['data'])

In [ ]:
similarities = cosine_similarity(vectorized)
print(similarities)


[[1.  0.  0.  ... 0.  0.  0. ]
 [0.  1.  0.5 ... 0.  0.  0. ]
 [0.  0.5 1.  ... 0.  0.  0. ]
 ...
 [0.  0.  0.  ... 1.  0.5 0.5]
 [0.  0.  0.  ... 0.5 1.  0.5]
 [0.  0.  0.  ... 0.5 0.5 1. ]]


In [ ]:
df_similarities = pd.DataFrame(similarities, columns=df_cleaned['Title'],index=df_cleaned['Title']).reset_index()
df_similarities

Title,Title,hereticalfishingbook1,thedungeonanarchistscookbook,dungeoncrawlercarl,carlsdoomsdayscenario,hereticalfishing2acozyguidetoannoyingthecultsoutsmartingthefishandalienatingoneselfunabridged,gideontheninth,hewhofightswithmonsters,hewhofightswithmonsters2,thebutchersmasquerade,...,houseofmanyways,dirkgentlysholisticdetectiveagency,thelongdarktea-timeofthesoul,thesalmonofdoubt,hereticalfishing3acozyguidetoannoyingthecultsoutsmartingthefishandalienatingoneself,somewherebeyondthesea,thehouseintheceruleansea,starwarsheirtotheempire,starwarsthelastcommand,starwarsdarkforcerising
0,hereticalfishingbook1,1.0,0.0,0.00,0.00,0.5,0.0,0.25,0.25,0.00,...,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0
1,thedungeonanarchistscookbook,0.0,1.0,0.50,0.50,0.0,0.0,0.00,0.00,0.50,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,dungeoncrawlercarl,0.0,0.5,1.00,0.75,0.0,0.0,0.00,0.00,0.75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,carlsdoomsdayscenario,0.0,0.5,0.75,1.00,0.0,0.0,0.00,0.00,0.75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,hereticalfishing2acozyguidetoannoyingthecultso...,0.5,0.0,0.00,0.00,1.0,0.0,0.25,0.25,0.00,...,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,somewherebeyondthesea,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,1.0,0.5,0.0,0.0,0.0
216,thehouseintheceruleansea,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.5,1.0,0.0,0.0,0.0
217,starwarsheirtotheempire,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.5,0.5
218,starwarsthelastcommand,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,1.0,0.5


In [ ]:
input_book = 'somewherebeyondthesea'
reccomendations = pd.DataFrame(df_similarities.nlargest(6,input_book)['Title'])
reccomendations = reccomendations[reccomendations['Title']!=input_book]
print(reccomendations)

                            Title
216      thehouseintheceruleansea
0           hereticalfishingbook1
1    thedungeonanarchistscookbook
2              dungeoncrawlercarl
3           carlsdoomsdayscenario
